引入包

In [1]:
from numpy import *
import re
import operator

# 使用Python进行文本分类

  词表到向量的转换函数

In [2]:
def loadDataSet():
    postingList=[['my','dog','has','flea', \
                 'problems','help','please'],
                ['maybe','not','take','him', \
                'to','dog','park','stupid'],
                 ['my','dalmation','is','so','cute', \
                 'I','love','him'],
                 ['stop','posting','stupid','worthless','garbage'],
                 ['mr','licks','ate','my','my','steak','how', \
                 'to','stop','him'],
                 ['quit','buying','worthlese','dog','food','stupid']]
    classVec = [0,1,0,1,0,1]   #1 代表侮辱性文字 0代表正常言论
    return postingList,classVec

#创建一个空集
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)  #创建两个集合的并集
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList) #创建一个其中所含元素都为0的向量
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word :%s in not in my Vocabulary" % word)
    return returnVec

In [3]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
print(myVocabList)

['help', 'park', 'dog', 'please', 'problems', 'take', 'so', 'worthless', 'how', 'food', 'buying', 'cute', 'dalmation', 'garbage', 'my', 'flea', 'stop', 'ate', 'quit', 'him', 'maybe', 'mr', 'steak', 'posting', 'love', 'worthlese', 'to', 'I', 'is', 'licks', 'stupid', 'not', 'has']


In [4]:
setOfWords2Vec(myVocabList,listOPosts[0])

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [5]:
setOfWords2Vec(myVocabList,listOPosts[3])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

朴素贝叶斯分类器训练函数

In [6]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    #p0Num = zeros(numWords);
    #p0Denom = 0.0
    #p1Num = zeros(numWords);
    #p1Denom = 0.0;
    p0Num = ones(numWords);
    p1Num = ones(numWords);
    p0Denom = 2.0;
    p1Denom = 2.0;
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    #p1Vect = p1Num/p1Denom           #change to log()
    #p0Vect = p0Num/p0Denom           #change to log()
    p0Vect = log(p0Num/p0Denom)
    p1Vect = log(p1Num/p1Denom)
    return p0Vect,p1Vect,pAbusive

In [7]:
listOPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0v,p1v,pAb = trainNB0(trainMat,listClasses)
print(pAb)

0.5


In [8]:
print(p0v)

[-2.56494936 -3.25809654 -2.56494936 -2.56494936 -2.56494936 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -1.87180218 -2.56494936 -2.56494936 -2.56494936
 -3.25809654 -2.15948425 -3.25809654 -2.56494936 -2.56494936 -3.25809654
 -2.56494936 -3.25809654 -2.56494936 -2.56494936 -2.56494936 -2.56494936
 -3.25809654 -3.25809654 -2.56494936]


In [9]:
print(p1v)

[-3.04452244 -2.35137526 -1.94591015 -3.04452244 -3.04452244 -2.35137526
 -3.04452244 -2.35137526 -3.04452244 -2.35137526 -2.35137526 -3.04452244
 -3.04452244 -2.35137526 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -2.35137526 -2.35137526 -2.35137526 -3.04452244 -3.04452244 -2.35137526
 -3.04452244 -2.35137526 -2.35137526 -3.04452244 -3.04452244 -3.04452244
 -1.65822808 -2.35137526 -3.04452244]


朴素贝叶斯分类函数

In [10]:
#元素相乘
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0v,p1v,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'Classified as:',classifyNB(thisDoc,p0v,p1v,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0v,p1v,pAb))

In [11]:
print(testingNB())

['love', 'my', 'dalmation'] Classified as: 0
['stupid', 'garbage'] classified as: 1
None


朴素贝叶斯词袋模型

In [12]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        return returnVec

# 使用个贝叶斯过滤垃圾邮件

In [13]:
mySent = 'This book is the best book on Python or M.L. I have every laid eyes upon'

In [14]:
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'every',
 'laid',
 'eyes',
 'upon']

In [15]:
regEx = re.compile('\\W*')
listOfTokens = regEx.split(mySent)
print(listOfTokens)

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'every', 'laid', 'eyes', 'upon']


D:\anaconda3\envs\mloldlab\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


In [16]:
[tok for tok in listOfTokens if len(tok)>0]

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M',
 'L',
 'I',
 'have',
 'every',
 'laid',
 'eyes',
 'upon']

In [17]:
[tok.lower() for tok in listOfTokens if len(tok)>0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'every',
 'laid',
 'eyes',
 'upon']

In [18]:
emailText = open('email/ham/6.txt').read()
listOfTokens = regEx.split(emailText)

D:\anaconda3\envs\mloldlab\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


In [19]:
def textParse(bigString):    #input is big string, #output is word list
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
    
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        #导出并解析文本文件
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50)); testSet=[]           #create test set
    #随机构建训练集
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    #对测试集进行分类
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error",docList[docIndex])
    print('the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText

In [20]:
spamTest()

D:\anaconda3\envs\mloldlab\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


classification error ['bargains', 'here', 'buy', 'phentermin', 'buy', 'genuine', 'phentermin', 'low', 'cost', 'visa', 'accepted', '130', '219', '292', '120', '366', '180', '513']
classification error ['bargains', 'here', 'buy', 'phentermin', 'buy', 'genuine', 'phentermin', 'low', 'cost', 'visa', 'accepted', '130', '219', '292', '120', '366', '180', '513']
classification error ['buy', 'ambiem', 'zolpidem', '5mg', '10mg', 'pill', 'pills', '129', 'pills', '199', '180', 'pills', '430', 'pills', '138', '120', 'pills', '322']
the error rate is:  0.3


# 朴素贝叶斯分类器从跟人广告中获取区域倾向

In [21]:
import feedparser as fp
ny = fp.parse("http://www.nasa.gov/rss/dyn/image_of_the_day.rss")

In [22]:
ny['entries']
len(ny['entries'])

60

In [23]:
def calcMostFreq(vocabList,fullText):
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:30]       

def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = list(range(2*minLen)); testSet=[]           #create test set
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList,p0v,p1v

In [24]:
ny = fp.parse("http://www.nasa.gov/rss/dyn/image_of_the_day.rss")
sf = fp.parse("http://sports.yahoo.com/nba/teams/hou/rss.xml")
vocabList,pSF,pNY=localWords(ny,sf)

the error rate is:  0.55


In [25]:
def getTopWords(ny,sf):
    vocabList,p0v,p1v = localWords(ny,sf)
    topNY=[]
    topSF=[]
    for i in range(len(p0v)):
        if p0v[i] > -6.0:
            topSF.append((vocabList[i],p0v[i]))
        if p1v[i] > -6.0:
            topSF.append((vocabList[i],p0v[i]))
        sortedSF = sorted(topSF,key=lambda pair: pair[1],reverse=True)
        print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
        for item in sortedSF:
            print(item[0])
        sortedNY = sorted(topNY,key=lambda pair: pair[1],reverse=True)
        print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
        for item in sortedNY:
            print(item[0])

In [26]:
getTopWords(ny,sf)

the error rate is:  0.4
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
shine
shine
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
structure
structure
shine
shine
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
structure
structure
https
https
shine
shine
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
structure
structure
https
https
two
two
shine
shine
NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
successful
successful
structure
structure
https
https
two
two
shine
shine
averages
averages
NY**NY*